# How to use
Run all and enter data when prompted

In [1]:
from torch.cuda import graph

# Get documents
documents = []

for i in range(int(input("Enter the number of documents for the AI to understand"))):
    documents.append(input("Copy the document here").replace("\n\n", ""))

documents

["Donkey Kong Country[b] is a 1994 platform game developed by Rare and published by Nintendo for the Super Nintendo Entertainment System (SNES). It is a reboot of Nintendo's Donkey Kong franchise and follows the gorilla Donkey Kong and his nephew Diddy Kong as they set out to recover their stolen banana hoard from the crocodile King K. Rool and his army, the Kremlings. The single-player traverses 40 side-scrolling levels as they jump between platforms and avoid obstacles. They collect items, ride minecarts and animals, defeat enemies and bosses, and find secret bonus stages. In multiplayer modes, two players work cooperatively or race.After developing Nintendo Entertainment System games in the 1980s, Rare, a British studio founded by Tim and Chris Stamper, purchased Silicon Graphics workstations to render 3D models. Nintendo sought a game to compete with Sega's Aladdin (1993) and commissioned Rare to revive the dormant Donkey Kong franchise. Rare assembled 12 developers to work on Donk

In [2]:
graph_gen = None

In [3]:
from GraphGen.graph_gen_integration import GraphGen
# Parse documents
triples = []
chunks = []

if graph_gen is None:
    graph_gen = GraphGen([], keyword_reward=1.05)


for idx, document in enumerate(documents):
    print(f"==================== Process document {idx+1}/{len(documents)} ====================")
    graph_gen.reset_model(triples, idx)
    triples += graph_gen.generate(document)
    chunks.append(graph_gen.split_chunks(document))
    
print("ACTUALLY DONE!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.11.5: Fast Gemma2 patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 30.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.5 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


==================== Process document 1/1 ====================
PROCESSING (1/13)
Phase 1 text chunk "Donkey Kong Country[b] is a 1994 platform game dev...army, the Kremlings."
Phase 1 context "None...None"
---
Phase 1 model start


AUTOTUNE bmm(16x182x256, 16x256x182)
  bmm 0.0133 ms 100.0% 
  triton_bmm_2 0.0184 ms 72.2% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=32, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=8
  triton_bmm_3 0.0184 ms 72.2% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=32, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=8
  triton_bmm_6 0.0184 ms 72.2% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_9 0.0195 ms 68.4% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=128, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=3, num_warps=4
  triton_bmm_10 0.0195 ms 68.4% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=128, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  trit

Phase 2 model generation done

Donkey_Kong_Country<unused1>publisher<unused2>Nintendo
Donkey_Kong_Country<unused1>starring<unused2>Diddy_Kong
Donkey_Kong_Country<unused1>world<unused2>Kongo
Donkey_Kong_Country<unused1>character<unused2>King_K._Rool
Donkey_Kong_Country<unused1>genre<unused2>Platform_game
Donkey_Kong_Country<unused1>developer<unused2>Rare
Donkey_Kong_Country<unused1>incident<unused2>Banana_hoard
Donkey_Kong_Country<unused1>platform<unused2>Super_Nintendo_Entertainment_System
Donkey_Kong_Country<unused1>yearOfPublication<unused2>1994
Phase 3 parsing done
chunk DONE
PROCESSING (2/13)
Phase 1 text chunk "It is a reboot of Nintendo's Donkey Kong franchise...army, the Kremlings."
Phase 1 context "<unused0>Donkey_Kong_Country<u...unused2>Banana_hoard"
---
Phase 1 model start
Phase 2 model generation done

Donkey_Kong_Country<unused1>followedBy<unused2>Donkey_Kong_64
Phase 3 parsing done
chunk DONE
PROCESSING (3/13)
Phase 1 text chunk "The single-player traverses 40 side-scroll

In [4]:
import pickle

with open('triples.smlgraph', 'wb') as outp:
    pickle.dump(triples, outp)
    
triples

[Triple(subject='Donkey_Kong_Country', predicate='publisher', object='Nintendo', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='starring', object='Diddy_Kong', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='world', object='Kongo', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='character', object='King_K._Rool', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='genre', object='Platform_game', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='developer', object='Rare', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='incident', object='Banana_hoard', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='platform', object='Super_Nintendo_Entertainment_System', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='yearOfPublication', object='1994', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predic

In [5]:
from pyvis.network import Network
from GraphGen.graph_gen_integration import Triple
from GraphGen import graph_gen_integration

net = Network(bgcolor="#222222", font_color="white", notebook=True, directed=True)
net2 = Network(bgcolor="#222222", font_color="white", notebook=False, directed=True)


# Parse rdf_strings

def add_triples(rdf: Triple, _color: str):
    net.add_node(rdf.subject, color=_color)
    net.add_node(rdf.object, color=_color)
    # if not any(edge['from'] == rdf.subject and edge['to'] == rdf.object and edge['title'] == rdf.predicate for edge in net.edges): # should be deprecated later
    net.add_edge(rdf.subject, rdf.object, title=rdf.predicate, color=_color)
    
    net2.add_node(rdf.subject, color=_color)
    net2.add_node(rdf.object, color=_color)
    # if not any(edge['from'] == rdf.subject and edge['to'] == rdf.object and edge['title'] == rdf.predicate for edge in net.edges): # should be deprecated later
    net2.add_edge(rdf.subject, rdf.object, title=rdf.predicate, color=_color)


import random

r = lambda: random.randint(0, 255)
net.toggle_physics(True)

colors = {}

for idx, triple in enumerate(triples):
    if triple.document_id in colors:
        document = colors[triple.document_id]
        if triple.id in document:
            color = document[triple.id]
        else:
            color = '#%02X%02X%02X' % (r(), r(), r())
            colors[triple.document_id][triple.id] = color
    else:
        colors[triple.document_id] = {} 
        color = '#%02X%02X%02X' % (r(), r(), r())
        colors[triple.document_id][triple.id] = color

    add_triples(triple, color)
    # net.show(f"{idx}.html", notebook=False)
net.show("graph.html", notebook=True)
# net2.show("graph2.html", notebook=False) # For visualization outside notebook

graph.html


In [6]:
# Workaround for now. Backtracking to source
def get_chunk(_hex_code):
    _hex_code = _hex_code.upper()
    print(_hex_code)
    possible_docs = []
    for _document in colors:
        for _chunk in colors[_document]:
            if colors[_document][_chunk] == _hex_code:
                possible_docs.append([_document, _chunk])
                
    if len(possible_docs) == 0:
        return None
    
    # print(possible_docs)
    
    docs = []
    
    for doc in possible_docs:
       docs.append(graph_gen.split_chunks(graph_gen.preprocess_document(documents[doc[0]]))[doc[1]])
    
    return docs

In [7]:
graph_gen.free_model()
del graph_gen

In [8]:
import torch, gc
# graph_gen.free_model()
# del graph_gen
gc.collect()
torch.cuda.empty_cache()

In [9]:
with open('triples.smlgraph', 'rb') as inp:
    triples = pickle.load(inp)
triples

[Triple(subject='Donkey_Kong_Country', predicate='publisher', object='Nintendo', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='starring', object='Diddy_Kong', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='world', object='Kongo', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='character', object='King_K._Rool', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='genre', object='Platform_game', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='developer', object='Rare', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='incident', object='Banana_hoard', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='platform', object='Super_Nintendo_Entertainment_System', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predicate='yearOfPublication', object='1994', id=0, document_id=0),
 Triple(subject='Donkey_Kong_Country', predic

In [10]:
from QueryGen.query_gen_integration import QueryGen
query_gen = QueryGen(triples)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like unsloth/gemma-2-2b-bnb-4bit is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [10]:
question = input("Enter your question:")

In [11]:
# question = input("Enter your question: ")
query = query_gen.generate(question)
# query
print(query)

A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
A
NOT STARTED
SELECT DISTINCT ?uri WHERE {?uri <purpose> <Federal_Communications_Commission>  . }


In [39]:
from rdflib import Graph, Literal, RDF, URIRef
g = Graph()
for triple in triples:
    g.add((URIRef(triple.subject), URIRef(triple.predicate), URIRef(triple.object)))

"Nationwide" does not look like a valid URI, trying to serialize this will break.
"Scholars' Bowl" does not look like a valid URI, trying to serialize this will break.
"Scholars' Bowl", "Academic Bowl", "Academic Team", "Academic Challenge", "Scholastic Bowl", "Primary School Quiz Bowl", "Middle School Quiz Bowl", "High School Quiz Bowl", "University Quiz Bowl" does not look like a valid URI, trying to serialize this will break.
"Nationwide" does not look like a valid URI, trying to serialize this will break.
"Unlimited" does not look like a valid URI, trying to serialize this will break.
"Unlimited" does not look like a valid URI, trying to serialize this will break.
"Single-elimination tournament" does not look like a valid URI, trying to serialize this will break.
"4 per team" does not look like a valid URI, trying to serialize this will break.
"Buzzer" does not look like a valid URI, trying to serialize this will break.
"1953-09-28" does not look like a valid URI, trying to seriali

In [40]:
# Apply the query to the graph and iterate through results
r_r = []
for r in g.query(query):
    print(r[0])
    r_r.append(str(r[0]))
r_r

[]

In [45]:
query_gen.free_model()
del query_gen
gc.collect()
torch.cuda.empty_cache()

NameError: name 'query_gen' is not defined

In [42]:
from AnswerGen.answer_gen_integration import AnswerGen
answer_gen = AnswerGen()

answer = answer_gen.generate(question, str(r_r))
answer

==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


'The quizbowl is the sport of the quiz bowl.<eos>'

In [46]:
answer_gen.free_model()
del answer_gen
gc.collect()
torch.cuda.empty_cache()

In [43]:
print(answer.replace("<eos>", ""))

The quizbowl is the sport of the quiz bowl.


In [ ]:
graph_gen = GraphGen([], keyword_reward=1.1)
graph_gen.reset_model([], 0)

In [8]:
# CHUNK LOOKUP
hex_code = input("Enter hex code for chunk lookup")
chunks = get_chunk(hex_code)
if chunks is None:
    print("No related chunk found. Try again")
elif len(chunks) == 1:
    print("CHUNK: ")
    print(chunks[0])
elif len(chunks) > 1:
    print(f"{len(chunks)} chunks somehow got assigned the same color. Find them below")
    for chunk in chunks:
        print(chunk)

#D55923
CHUNK: 
Will Wood is an American musician, singer-songwriter, and comedian. [4][5] Wood has released four studio albums; Everything Is a Lot (2015), SELF-iSH (2016), The Normal Album (2020),[6] and "In case I make it," (2022). The former two were released as Will Wood and the Tapeworms, Wood's prior band name. He has additionally released two live albums and a soundtrack.


In [50]:
colors

{0: {0: '#2B6E1E', 1: '#A29881', 2: '#74686B'},
 1: {0: '#355423', 1: '#752F2E', 2: '#A19E54', 3: '#B0681C', 4: '#B5BCC9'}}